# German-Power-Plants Workflow

## Data Origin
source\[0\]: conventional power plants in germany:
https://data.open-power-system-data.org/conventional_power_plants/2018-12-20/conventional_power_plants_DE.csv<br/>
source\[1\]: GeoJSON ploygons for German districts:
https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/master/4_kreise/2_hoch.geo.json

## Workflow
### aim(data):
- produce output.geojson, combining both data sources, mapping the sum of power to each polygon.  -- [download output.geojso](https://workflowkdcc.s3-eu-west-1.amazonaws.com/germanyenergy/output.geojson).


### aim(science):
- visualize total, renewable, fossil and nuclear power for german district.


### structure:
- [KDSC/workflow/germanen.yaml](https://github.com/IdontApply/KDSC/blob/master/KDSC/workflow/germanen.yaml)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from IPython.display import display as di
import geopy
import folium
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim
import geopandas as gpd
import json
import os

from branca import colormap as cm

                         

In [ ]:

prefix = 'germanyenergy/'

os.system('gsutil cp gs://workflowkddc/germanen/output.geojson output.geojson')
geo_data=r"./output.geojson"

def cholo( ele, geo_data, color):
    
    de = gpd.read_file(geo_data,)
    
    ele = de.set_index('ID_3')[ele]
    colormap = color
    # ,vmin=0,vmax=8000)
    
    m = folium.Map(
        location=[51.1657, 10.4515],
        zoom_start=6,
        min_zoom=6, 
        max_zoom=6
    )
    folium.GeoJson(
        geo_data,
        name='germany init drawing',
        style_function=lambda feature: {
            'fillColor': colormap(ele[feature['properties']['ID_3']]),
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillOpacity': 0.9,
    }
).add_to(m)
    
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    NIL = folium.features.GeoJson(
        "./output.geojson",
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NAME_3','total','ID_3'],
            aliases=['Neighborhood: ','Resident foreign population in %: ','ID:  '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(NIL)
    m.keep_in_front(NIL)
    folium.LayerControl().add_to(m)
    display(m)


cholo('total', geo_data, cm.LinearColormap(['white' ,'yellow', 'orange', 'red'],index=[0, 100, 1000, 8000],vmin=0,vmax=8000))

In [ ]:


'Renewable energy'
'Nuclear'
'Fossil fuels'
cholo('Nuclear',  geo_data, cm.LinearColormap(['white' ,'lightblue', 'blue','darkblue'],index=[0, 100, 1000, 8000],vmin=0,vmax=8000))


In [ ]:
cholo( 'Fossil fuels',  geo_data, cm.LinearColormap(['white' ,'lightgray', 'gray','black'],index=[0, 100, 1000, 8000],vmin=0,vmax=8000))

In [ ]:
cholo( 'Renewable energy',  geo_data, cm.LinearColormap(['white' ,'lime', 'green','darkgreen'],index=[0, 100, 1000, 8000],vmin=0,vmax=8000))
